In [1]:
import speech_recognition as sr # pip install SpeechRecognition
import pyaudio
import webbrowser

In [2]:
def gravar_audio(nome_arquivo="audio.wav", duracao=5):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Fale alguma coisa...")
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source, phrase_time_limit=duracao)

    with open(nome_arquivo, "wb") as f:
        f.write(audio.get_wav_data())
        
    print(f"Áudio gravado em {nome_arquivo}")
    return nome_arquivo

In [3]:
def audio_para_texto(nome_arquivo):
    r = sr.Recognizer()
    with sr.AudioFile(nome_arquivo) as source:
        audio = r.record(source)

    try:
        texto = r.recognize_google(audio, language='pt-BR')
        print(f"Texto reconhecido: {texto}")
        return texto
    except sr.UnknownValueError:
        print("Não foi possível entender o áudio.")
        return None
    except sr.RequestError as e:
        print(f"Erro ao solicitar resultados do serviço de reconhecimento de fala do Google; {e}")
        return None

In [4]:
INTENCOES_E_PALAVRAS_CHAVE = {
    "pesquisar": ["pesquisar", "buscar", "procurar", "encontrar"],
    "tocar_media": ["tocar", "ouvir", "reproduzir"]
}

In [5]:
def extrair_intencao_e_objeto(texto):
    texto_original = texto.lower()
    intencao_identificada = "desconhecida"
    objeto_identificado = None

    # Vamos iterar pelas intenções e suas palavras-chave
    for intencao_nome, palavras_chave in INTENCOES_E_PALAVRAS_CHAVE.items():
        for palavra_chave in palavras_chave:
            # Usamos find para saber se a palavra chave está no texto
            indice = texto_original.find(palavra_chave)

            if indice != -1:
                # Encontramos uma palavra chave que define a intenção
                intencao_identificada = intencao_nome

                # Removemos a palavra chave da intenção do texto para extrair o objeto
                # Usamos replace com count=1 para remover apenas a primeira ocorrência
                texto_apos_remocao = texto_original.replace(palavra_chave, "").strip()

                # Limpeza básica do objeto: remove pontuações e normaliza espaços
                objeto_limpo = ""
                for char in texto_apos_remocao:
                    if char.isalnum() or char.isspace():
                        objeto_limpo += char
                objeto_limpo = ' '.join(objeto_limpo.split()) # Normaliza espaços

                if objeto_limpo:
                    objeto_identificado = objeto_limpo

                # Se encontramos uma intenção e um objeto, podemos parar a busca por intenções
                if intencao_identificada != "desconhecida" and objeto_identificado:
                    break # Sai do loop de palavras_chave
        if intencao_identificada != "desconhecida" and objeto_identificado:
            break # Sai do loop de intenções

    return {"intencao": intencao_identificada, "objeto": objeto_identificado}

In [6]:
def executar_acao(resultado_nlp):
    intencao = resultado_nlp["intencao"]
    objeto = resultado_nlp["objeto"]

    # --- Execução das Ações ---
    if intencao == "pesquisar":
        if objeto:
            print(f"Executando pesquisa no Google por: '{objeto}'")
            try:
                webbrowser.open(f"https://www.google.com/search?q={objeto}")
            except Exception as e:
                print(f"Não foi possível abrir o navegador: {e}")
        else:
            print("Não sei o que pesquisar.")
    elif intencao == "tocar_media":
        if objeto:
            print(f"Executando reprodução no Youtube de: '{objeto}'")
            try:
                webbrowser.open(f"https://www.youtube.com/results?search_query={objeto}")
            except Exception as e:
                print(f"Não foi possível abrir o Youtube: {e}")
        else:
            print("Não sei o que tocar.")
    else:
        print("Comando não compreendido.")

In [19]:
arquivo_audio = gravar_audio("comando_voz.wav", duracao=8)

Fale alguma coisa...
Áudio gravado em comando_voz.wav


In [20]:
texto_do_comando = audio_para_texto("comando_voz.wav")

Texto reconhecido: encontrar farmácia mais próxima


In [21]:
if texto_do_comando:
    resultado_nlp = extrair_intencao_e_objeto(texto_do_comando)
    print(f"Interpretado: {resultado_nlp}")
    executar_acao(resultado_nlp)

Interpretado: {'intencao': 'pesquisar', 'objeto': 'farmácia mais próxima'}
Executando pesquisa no Google por: 'farmácia mais próxima'
